In [1]:
import os
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/atharvmohanjadhav/DS-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "atharvmohanjadhav"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "4a58f038508d5a342bdb03ad3effba338e994414"

In [2]:
import os
%pwd

'd:\\MLOps Udemy Krish Naik\\DS-Project\\research'

In [3]:
os.chdir("../")
%pwd

'd:\\MLOps Udemy Krish Naik\\DS-Project'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metrics_file_path: Path
    target_column: str
    mlflow_uri:str

In [9]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml,create_dirs,save_json
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_fileppath=PARAMS_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_fileppath)
        self.schema=read_yaml(schema_filepath)

        create_dirs([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_dirs([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir= config.root_dir,
            test_data_path= config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metrics_file_path=config.metrics_file_path,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/atharvmohanjadhav/DS-Project.mlflow"
        )
        return model_evaluation_config



In [18]:
import os
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
import pandas as pd

class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual,pred):
        rmse = np.sqrt(mean_squared_error(actual,pred))
        mae = mean_absolute_error(actual,pred)
        r2 = r2_score(actual,pred)
        return rmse,mae,r2

In [30]:
import os
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
import pandas as pd

class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual,pred):
        rmse = np.sqrt(mean_squared_error(actual,pred))
        mae = mean_absolute_error(actual,pred)
        r2 = r2_score(actual,pred)
        return rmse,mae,r2

    def log_mlflow(self):
        test_df = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        x_test = test_df.drop([self.config.target_column],axis=1)
        y_test = test_df[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_uri_type = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            y_pred = model.predict(x_test)

            (rmse,mae,r2) = self.eval_metrics(y_test,y_pred)
            scores = {"rmse":rmse,"mae":mae,"r2":r2}
            save_json(path=Path(self.config.metrics_file_path),data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse",rmse)
            mlflow.log_metric("mae",mae)
            mlflow.log_metric("r2",r2)

            if tracking_uri_type != "file":
                mlflow.sklearn.log_model(model,"model",registered_model_name="ElasticNetModel")
            else:
                mlflow.sklearn.log_model(model,"model")




In [31]:
try:
    config = ConfigurationManager()
    mode_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=mode_evaluation_config)
    model_evaluation.log_mlflow()
except Exception as e:
    raise e

[2025-03-30 13:20:42,488:INFO: common: yaml file: <_io.TextIOWrapper name='config\\config.yaml' mode='r' encoding='UTF-8'> loaded succesfully!]
[2025-03-30 13:20:42,494:INFO: common: yaml file: <_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> loaded succesfully!]
[2025-03-30 13:20:42,499:INFO: common: yaml file: <_io.TextIOWrapper name='schema.yaml' mode='r' encoding='UTF-8'> loaded succesfully!]
[2025-03-30 13:20:42,502:INFO: common: created directory at: artifacts]
[2025-03-30 13:20:42,504:INFO: common: created directory at: artifacts/model_evaluation]
[2025-03-30 13:21:05,563:WARNING: connectionpool: Retrying (Retry(total=6, connect=7, read=6, redirect=7, status=7)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /atharvmohanjadhav/DS-Project.mlflow/api/2.0/mlflow/runs/create]
[2025-03-30 13:21:06,741:INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


2025/03/30 13:21:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticNetModel'.
2025/03/30 13:21:24 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticNetModel, version 1
Created version '1' of model 'ElasticNetModel'.


🏃 View run thundering-mink-954 at: https://dagshub.com/atharvmohanjadhav/DS-Project.mlflow/#/experiments/0/runs/dcc039fa9d2f4becb0c7fa54ecde18a1
🧪 View experiment at: https://dagshub.com/atharvmohanjadhav/DS-Project.mlflow/#/experiments/0
